# 📐 Measuring discrimination with linear regression

❗❗❗ **Make sure to save a copy of this notebook to your Google Drive so your work isn't lost.**

## Introduction

In this tutorial, we'll use `R` to fit linear models that assess potentially discriminatory behavior in a hypothetical federal pretrial setting.

By the end of the tutorial, you'll have foundational understanding of the following:
1. 📏 How linear regression can be used to assess potential discrimination.
2. 📈 How to fit and interpret a linear model using `R`.

## ✅ Set up

Make sure to run the cell below. It imports additional useful functions, adjusts `R` settings, and loads in data. 

In [0]:
# Load in additional functions
library(tidyverse)

# Use three digits past the decimal point
options(digits = 3)

# This makes our plots look nice!
theme_set(theme_bw())

# This is where the data is stored.
DATA_PATH = "https://github.com/joshuagrossman/mse298-labs/raw/main/week3/data/lab-3-data.rds"

# Load in the data
data = read_rds(DATA_PATH)

## 🏛️ Primer on federal pretrial incarceration

⏮️ In the last lab, we covered the basics of the criminal pretrial process. 

⏭️ In this lab, we specifically consider federal pretrial detention.

⚖️ In several federal jurisdictions (though not all!), the pretrial process proceeds as follows:
1. First, there is an **initial hearing**.
2. During the initial hearing, the prosecutor may recommend that the defendant should be detained before their trial (i.e., the prosecutor **"moves to detain"**).
3. If the prosecutor does not move to detain, the defendant is typically released at the initial hearing.
4. If the prosecutor moves to detain, the defendant is temporarily detained. Officers of the court take several days to learn more about the defendant.
5. After this investigation, there is a **detention hearing**.
6. At the detention hearing, the judge decides whether the defendant should be detained or released before their trial. 

🎓 For more context on the federal pretrial process, see [Grossman, Nyarko, Goel (2023)](https://5harad.com/papers/multi-stage-bias.pdf).

This lab will analyze potential racial bias in **hypothetical motions for detention among federal prosecutors**.

## 🚀 Exercise: Understanding the data

The data in this lab is from a hypothetical pretrial setting. Each row represents a defendant.

1. Inspect the data. Which column indicates whether the prosecution moved to detain?

2. Using the data, calculate the proportion of defendants for whom the prosecution moves for detention. 

3. Then, calculate the same proportion for each race and ethnicity group. What do you find?

In [0]:
# Your code here!



# 🗺️ The world's fastest primer on linear regression

**Linear regression estimates conditional means.** 

🤝 For example, I may be interested in estimating the hiring rate for different combinations of gender, race, and age.

Similar to the previous exercise, you could use a `group_by` and `summarize` statement to calculate the observed hiring rate for each combination.

👎 But, this approach has several downsides. For example:
- You need a table with $g*r*a$ cells to communicate the results, where $g$ is the number of genders, $r$ is the number of race groups, and $a$ is the number of age groups.
- How do I estimate hiring rates for cells with no data? For example, what if my sample has no 35 year-old Asian women?
- Can I trust estimated hiring rates in cells with only a few data points, or with big outliers?

〰️ Linear regression allows you to "smooth over" the observed combinations, and efficiently describe patterns in the data.

> That being said, in many scenarios, it can be informative to estimate and visualize conditional means directly (i.e., without using linear regression). We do this in the next exercise.

# 🚀 Exercise: Visualizing linear regression

1. For each value of the `risk` column, calculate the rate at which the prosecution moves for detention, and also the size of each group.

> `risk` is the defendant's statistically-estimated probability of violating if released. Later in the course, we'll discuss risk estimation.
> 
> The `n()` function may come in handy.

2. Using `ggplot()` to plot, for each value of `risk`, the rate at which the prosecution moves for detention. Map the size of the points to the size of each group, and connect the points with a line. 

3. Add a regression line to your plot with the following code:

> `+ geom_smooth(data=data, mapping=aes(x=risk, y=prosecution_detain), method='lm')`
>
> where `data` is the original data we loaded in above. `lm` stands for linear model.

Does the regression line appear to fit the data well? Explain.

In [0]:
# Your code here!



## 🚀 Exercise: Connecting the output of `lm` to conditional means

Re-run your code above that calculates, by race, the proportion of defendants for whom the prosecution moves for detention. 

Then, run the following:

`lm(prosecution_detain ~ 1 + race, data=data)`

Are you able to connect the proportions you calculated to the output of this command? Explain.

In [0]:
# Your code here!



## 🧮 Regression formulas

The linear regression algorithm fits a model to the data that estimates the **average value of the outcome conditional on the input variables**.

Using only the information in the `race` column, the algorithm made its best guess for the average value of `prosecution_detain` (i.e., the probability of moving for detention) for each possible value in the `race` column.

🔢 Here's the output as a formula:

$\widehat{\text{prosecution_detain}}_i = 0.451 + 0.295*\mathbb{I}_{\text{raceB}, i} + 0.282*\mathbb{I}_{\text{raceH}, i}$,

where $\widehat{\text{prosecution_detain}}_i$ is the estimated probability of $\text{prosecution_detain}$ for individual $i$, $\mathbb{I}_{\text{raceB}, i}$ is an indicator of whether individual $i$ is Black, and $\mathbb{I}_{\text{raceH}, i}$ is an indicator of whether individual $i$ is Hispanic.

> For reasons beyond the scope of this tutorial, the `lm` function automatically designates a **base level** of race/ethnicity, which in this case is `W` (i.e., white).
This base level is represented by $0.451$, or the **intercept term**.

## ➕ Including additional variables

Regression is not limited to one input. For example, we can include both the `race` and `risk` columns:

In [0]:
lm(prosecution_detain ~ 1 + race + risk, data=data)

🔎 Here's how to interpret these values:
1. `raceB` : **On average**, the prosecution is 11 percentage points more likely to move for detention of Black defendants than white defendants **with the same risk level**.
2. `raceH` : **On average**, the prosecution is 18 percentage points more likely to move for detention of Hispanic defendants than white defendants **with the same risk level**.
3. `risk` : A one point increase in risk leads to an 8 percentage point increase in the **estimated** probability of a motion for detention, **holding all other variables (i.e., race) constant**.
4. `(Intercept)` : For a white defendant with a risk level of 0, the prosecution moves for detention with an estimated probability of 0.28.

> ❗❗❗ Percentage points are different than percentages. For example, suppose my chance of winning a prize is 60%. If my chance of winning drops by 50%, my new probability of winning is **30%**. If my chance of winning drops by 50 percentage points, my new probability is **10%**.

👉 It looks like the racial disparities in the rate of moving for detention dropped substantially once we **adjusted** for estimated risk. 

> But, estimated risk isn't the only factor a prosecutor might account for in their decision to detain. For example, prosecuctors may be statutorily compelled to move for detention if the defendant is charged with certain crimes.

## 🚀 Exercise: Running a "kitchen-sink" regression

Fit a regression model that includes all columns in the data **except for `released`**, which indicates whether the defendant was **released at the initial hearing**.

Here's how to fit the model:

`lm(prosecution_detain ~ . - released, data=data)`

How do you interpret the `raceB` and `raceH` coefficients?

In [0]:
# Your code here!



## 🚀 Exercise: What about the `released` column?

Fit a regression model that includes **all** columns in the data, including `released`.

Why do you think the `raceH` coefficient changed so drastically?

> Hint: Create a confusion matrix for `prosecution_detain` and `released`. Remember that `released` indicates whether an individual was released at the initial hearing.

In [0]:
# Your code here!

